Replicating the 3 figures (Training task 4)


In [3]:
import neuro_tracer as nt

import sys,copy,os,inspect

neurovis_path = os.path.abspath(r'D:\projectome_analysis\neuron-vis\neuronVis')
sys.path.append(neurovis_path)

import IONData 
iondata = IONData.IONData()

from collections import defaultdict


from pathlib import Path
import matplotlib


import matplotlib.pyplot as plt


import numpy as np
import nibabel as nib  # parse the NII data
from collections import deque
import pandas as pd
from typing import Tuple
from io import StringIO

import seaborn as sns
from sklearn.cluster import AgglomerativeClustering
import matplotlib.cm as cm

from scipy.cluster.hierarchy import dendrogram, linkage

import requests
from bs4 import BeautifulSoup
neuronlist=iondata.getNeuronListBySampleID('251637')


c:\Users\binbi\anaconda3\envs\neuronVis\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\binbi\anaconda3\envs\neuronVis\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\binbi\anaconda3\envs\neuronVis\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Workflow:

Step 1: load the template (Charm) and regional neuron swc (using neuro_tracer module)
Step 1.1: inspect the template nifti and its annotation file, get the basic understanding.

"D:\Monkey_projectome_training\python-learn\resources\atlas\NMT_v2.0_sym\NMT_v2.0_sym\ARM_in_NMT_v2.1_sym.nii.gz"


In [4]:

atlas_base_path = Path(r'D:\Monkey_projectome_training\resources\atlas\IMT')
D99_atlas_path = atlas_base_path / r"IMT_D99ParcellationSym.nii"
D99_atlas= nib.load(D99_atlas_path)     
SARM_atlas_path = atlas_base_path / r"IMT_SARM.nii"
SARM_atlas= nib.load(SARM_atlas_path)

global_id_df = pd.read_excel(atlas_base_path / 'D99+SARM_labeltable.xlsx')

D99_voxels = D99_atlas.get_fdata()
SARM_voxels = SARM_atlas.get_fdata()
print(D99_voxels.shape,\
SARM_voxels.shape)
D99_atlas.header.get_zooms()

(800, 1000, 680) (800, 1000, 680)


(0.075, 0.075, 0.075)

In [ ]:



target_neuron_list = iondata.getNeuronListBySampleID('251637')


In [ ]:
class region_analysis_per_neuron:
    def __init__(self, neuron_tracer_obj,atlas,atlas_table):
            self.neuron = neuron_tracer_obj
            self.exp_name = neuron_tracer_obj.exp_no
            self.atlas = atlas
          
            self.atlas_table = atlas_table
            self.brain_region_map = {index: row['Abbreviation'] for index, row in self.atlas_table.iterrows()}

    
    def region_analysis(self):

        self.mapped_brain_region_lengths, self.neuron_total_length = self._calculate_neuronal_branch_length()
        self.soma_region, self.terminal_regions = self._soma_and_terminal_region()
    def _distance(self,p1, p2,space='nii'):

        if space == 'nii':
            return np.sqrt((p1.x_nii - p2.x_nii)**2 + (p1.y_nii - p2.y_nii)**2 + (p1.z_nii - p2.z_nii)**2)        
        elif space == 'native':
            return np.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2 + (p1.z - p2.z)**2)
        else:
            raise ValueError(f"Invalid space: {space}")
    
    
    def _calculate_neuronal_branch_length(self):
            
            brain_region_lengths = defaultdict(float)
            neuron_total_length = 0
            
            for branch in self.neuron.branches:
                branch_length = 0
                current_node = branch[0]
                
                node_index = 0
                while node_index < len(branch) - 1:
                    current_node = branch[node_index]
                    child_id = node_index + 1
                    child_node = branch[child_id]
                    edge_length = round(self._distance(current_node, child_node),3)
                    aa = tuple(np.round(np.array([current_node.x_nii, current_node.y_nii, current_node.z_nii])).astype(int).flatten())
                    branch_length += edge_length

        
                    
                    if (0 <= aa[0] < self.atlas.shape[0] and
                        0 <= aa[1] < self.atlas.shape[1] and
                        0 <= aa[2] < self.atlas.shape[2]):

                        brain_region_id = int(self.atlas[aa])
                        brain_region_lengths[brain_region_id] += edge_length
                    else:
                        print(f"Warning: Point {aa},{[current_node.x_nii, current_node.y_nii, current_node.z_nii]} {self.neuron.swc_filename} node{current_node.id} is out of bounds of the atlas.")
                    node_index += 1
                neuron_total_length += branch_length
         
            mapped_brain_region_lengths ={}

            for index, length in brain_region_lengths.items():
               
                region = self.brain_region_map.get(index, 'Unknown')
                
                mapped_brain_region_lengths [region] = length
                
            return mapped_brain_region_lengths, neuron_total_length
    def _soma_and_terminal_region(self):
        soma=self.neuron.root
        soma_pos=tuple(np.round(np.array([soma.x_nii, soma.y_nii, soma.z_nii])).astype(int).flatten())
        soma_region = self.brain_region_map.get(int(self.atlas[soma_pos]), 'Unknown')
        
        terminal_regions = {}
        for node in self.neuron.terminal_nodes:
            node_pos=tuple(np.round(np.array([node.x_nii, node.y_nii, node.z_nii])).astype(int).flatten())
            node_region = int(self.atlas[node_pos])
            node_region = self.brain_region_map.get(node_region, 'Unknown')
            
            terminal_regions [node_region] = node_pos
        return soma_region, terminal_regions   


In [ ]:

neurons_of_experiment={}
combined_atlas = np.where(D99_voxels != 0, D99_voxels, SARM_voxels)

soma_regions = []
neuronIDs = []
projection_lengths = []
terminal_regions = []
region_projection_lengths =[]
for target_neuron in target_neuron_list:
    neuron = nt.neuro_tracer()
    neuron.process(target_neuron['sampleid'],target_neuron['name'],nii_space='monkey')
    neurons_of_experiment[int(target_neuron['name'].split('.')[0])] = neuron
    
    target_neuron_projection= region_analysis_per_neuron(neuron,combined_atlas,global_id_df)
    target_neuron_projection.region_analysis()
    projection_lengths.append(target_neuron_projection.neuron_total_length)
    soma_regions.append(target_neuron_projection.soma_region)
    neuronIDs.append(neuron.exp_no+'_'+neuron.swc_filename)
    terminal_regions.append(list(target_neuron_projection.terminal_regions.keys()))

    region_projection_lengths.append(target_neuron_projection.mapped_brain_region_lengths)
    # swc2tree = Swc2tree()
    # swc2tree.get_nodes(data.values)
    # swc2tree.get_special_nodes()
    # swc2tree.get_edges()
    
    # neuron_analysis = Neuron_analysis_nii(atla_id = atla_id)
    # neuron_analysis.transform_swc2nii(swc2tree.swc,res,nii_shape)
    # neuron_analysis.get_somalocation(nii_label)
    # neuron_analysis.get_dendrite_length(swc2tree.edges,nii_label)

    # somaroi.append(neuron_analysis.somaroi)
    # somaname.append(neurons['name'])
    # dendrite_length.append(list(neuron_analysis.dendrite_length.values()))

In [ ]:
%matplotlib inline
neuron = nt.neuro_tracer()

neuron.process('251637','157.swc',nii_space='monkey')
target_neuron_projection= region_analysis_per_neuron(neuron,combined_atlas,global_id_df)
target_neuron_projection.region_analysis()

target_neuron_projection.neuron.plot_neuron()
# print(target_neuron_projection.neuron.nodes[1439].x_nii)
# print(target_neuron_projection.neuron.nodes[1439].y_nii)
# print(target_neuron_projection.neuron.nodes[1439].z_nii)



(692, 683, 91)


In [ ]:
aa = tuple(np.round(np.array([current_node.x_nii, current_node.y_nii, current_node.z_nii])).astype(int).flatten())

NameError: name 'current_node' is not defined

In [21]:
target_neuron_projection.neuron.nodes[1].__dict__.id

AttributeError: 'dict' object has no attribute 'id'

In [6]:
combined_atlas = np.where(D99_voxels != 0, D99_voxels, SARM_voxels)


In [ ]:

figure_data = {
    'NeuronID': neuronIDs,
    'Soma_Region': soma_regions,
    'Projection_length':projection_lengths,
    'Region_projection_length':region_projection_lengths
}


figure_data=pd.DataFrame(figure_data)

figure1=figure_data.copy(deep=True)
plt.figure(figsize=(10, 6))
plt.boxplot([figure1[figure1['Soma_Region'] == region]['Projection_length'] for region in figure1['Soma_Region'].unique()],
            labels=figure1['Soma_Region'].unique())

# Add titles and labels
plt.title('Box Plot of Projection Length by Soma Region')
plt.xlabel('Soma Regions')
plt.ylabel('Projection Length')

# Show the plot
plt.xticks(rotation=45) 
plt.tight_layout()  
plt.show()

In [ ]:
%matplotlib inline
figure2 = pd.DataFrame(region_projection_lengths,index=neuronIDs)
figure2.fillna(0,inplace=True)


row_linkage = linkage(figure2.T, method='ward', metric='euclidean')  # Cluster neurons

g = sns.clustermap(figure2.T,
                   row_linkage=row_linkage,
                   row_cluster=False)

g.ax_heatmap.set_ylabel('Brain Regions')
g.ax_heatmap.set_xlabel('Neurons')


# Show the plot
plt.show()

In [ ]:
figure3 = pd.DataFrame(soma_regions,index=neuronIDs)
